# DTOcean Installation Module Test

In [25]:
%matplotlib inline

In [26]:
from IPython.display import display, HTML

In [27]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [28]:
import numpy as np

In [29]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu, ThemeMenu
from dtocean_core.pipeline import Tree

In [30]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [31]:
# Bring up the logger
start_logging()

2016-12-18 14:54:45,071 - INFO - dtocean_core - Begin logging for dtocean_core.


## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [32]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
theme_menu = ThemeMenu()
pipe_tree = Tree()

## Create a new project

In [33]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

2016-12-18 14:54:46,461 - INFO - aneris.entity.simulation - Created new Simulation with title "Default"
2016-12-18 14:54:46,463 - INFO - aneris.control.simulation - Datastate with level "initial" stored
2016-12-18 14:54:46,463 - INFO - aneris.control.pipeline - New Hub created for interface ProjectInterface.


## Set the device type

In [34]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type"
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Wave Floating")
my_var.read(new_core, new_project)

2016-12-18 14:54:46,490 - INFO - aneris.control.data - New "device.system_type" data stored with index N8UU6P
2016-12-18 14:54:46,492 - INFO - aneris.control.simulation - Datastate stored


## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [35]:
project_menu.initiate_pipeline(new_core, new_project)

2016-12-18 14:54:46,509 - INFO - aneris.control.simulation - Datastate with level "system type selection start" stored
2016-12-18 14:54:46,512 - INFO - aneris.control.data - New "hidden.pipeline_active" data stored with index DN2B8X
2016-12-18 14:54:46,513 - INFO - aneris.control.simulation - Datastate with level "system type selection output" stored
2016-12-18 14:54:46,516 - INFO - aneris.control.pipeline - New Pipeline created for interface ModuleInterface.
2016-12-18 14:54:46,516 - INFO - aneris.control.pipeline - New Hub created for interface ThemeInterface.


## Discover available modules

In [36]:
names = module_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

## Activate a module

Note that the order of activation is important and that we can't deactivate yet!

In [37]:
module_name = 'Installation'
module_menu.activate(new_core, new_project, module_name)

## Activate the Economics themes

In [38]:
names = theme_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

In [39]:
theme_menu.activate(new_core, new_project, "Economics")

## Check the status of the module inputs

In [40]:
installation_branch = pipe_tree.get_branch(new_core, new_project, 'Installation')
input_status = installation_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [41]:
project_menu.initiate_dataflow(new_core, new_project)

2016-12-18 14:54:46,667 - INFO - aneris.control.data - New "hidden.dataflow_active" data stored with index 6YMGA2
2016-12-18 14:54:46,667 - INFO - aneris.control.simulation - Datastate stored
2016-12-18 14:54:46,674 - INFO - aneris.control.simulation - Datastate with level "modules initial" stored


## Load test data

Prepare the test data for loading. The files required can be found in the test_data directory of the source code and should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [42]:
%run inputs_wp5.py

generate test data: D:\Usuarios\108630\Programming\Python\notebooks\inputs_wp5.pkl


In [43]:
installation_branch.read_test_data(new_core,
                            new_project,
                            "inputs_wp5.pkl")

2016-12-18 14:55:05,825 - INFO - aneris.control.data - New "component.power_quality" data stored with index RINPEV
2016-12-18 14:55:05,828 - INFO - aneris.control.data - New "farm.electrical_component_data" data stored with index X6K9I3
2016-12-18 14:55:05,829 - INFO - aneris.control.data - New "device.bollard_pull" data stored with index EI8CAJ
2016-12-18 14:55:05,831 - INFO - aneris.control.data - New "device.transportation_method" data stored with index KT2BC6
2016-12-18 14:55:05,832 - INFO - aneris.control.data - New "component.switchgear" data stored with index IUTUZK
2016-12-18 14:55:05,835 - INFO - aneris.control.data - New "component.collection_points" data stored with index C4ONGB
2016-12-18 14:55:05,838 - INFO - aneris.control.data - New "project.port_percentage_cost" data stored with index SAPEGK
2016-12-18 14:55:06,329 - INFO - aneris.control.data - New "farm.wind_series_installation" data stored with index 48ROFJ
2016-12-18 14:55:06,332 - INFO - aneris.control.data - New "

## Prepare the Economics Theme

In [44]:
theme_name = "Economics"
eco_branch = pipe_tree.get_branch(new_core, new_project, "Economics")

input_status = eco_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Set the discount rate and cost estimates and project lifetime

In [45]:
discount_rate = 0.05
device_cost = 1000000
n_devices = 2
project_lifetime = 20

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "project.discount_rate")
new_var.set_raw_interface(new_core, discount_rate)
new_var.read(new_core, new_project)

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "device.system_cost")
new_var.set_raw_interface(new_core, device_cost)
new_var.read(new_core, new_project)

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "farm.number_of_devices")
new_var.set_raw_interface(new_core, n_devices)
new_var.read(new_core, new_project)

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "project.lifetime")
new_var.set_raw_interface(new_core, project_lifetime)
new_var.read(new_core, new_project)

2016-12-18 14:55:07,401 - INFO - aneris.control.data - New "project.discount_rate" data stored with index M2HPY0
2016-12-18 14:55:07,401 - INFO - aneris.control.simulation - Datastate stored
2016-12-18 14:55:07,417 - INFO - aneris.control.data - New "device.system_cost" data stored with index EXKO7R
2016-12-18 14:55:07,417 - INFO - aneris.control.simulation - Datastate stored
2016-12-18 14:55:07,430 - INFO - aneris.control.data - New "farm.number_of_devices" data stored with index Y4CU6X
2016-12-18 14:55:07,430 - INFO - aneris.control.simulation - Datastate stored
2016-12-18 14:55:07,441 - INFO - aneris.control.data - New "project.lifetime" data stored with index 1Z99GP
2016-12-18 14:55:07,443 - INFO - aneris.control.simulation - Datastate stored


## Check if the module can be executed

In [46]:
can_execute = module_menu.is_executable(new_core, new_project, module_name)
display(can_execute)

True

In [47]:
input_status = installation_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [48]:
module_menu.execute_current(new_core, new_project)

2016-12-18 14:55:07,487 - INFO - aneris.control.simulation - Datastate with level "installation start" stored
2016-12-18 14:55:08,111 - INFO - dtocean_installation.main - Installation port selected...
2016-12-18 14:55:08,118 - WARNING - dtocean_logistics.phases.install.e_export - Export cable trenching technique not valid. A plough is assumed.
2016-12-18 14:55:08,141 - WARNING - dtocean_logistics.phases.install.e_array - Array cable trenching technique not valid.A plough is assumed.
2016-12-18 14:55:08,207 - WARNING - dtocean_logistics.phases.install.Driven - Drilling technique inadequate for site soil types.
2016-12-18 14:55:08,227 - INFO - dtocean_installation.main - Checking installation requirements for phase: Installation of driven piles anchors/foundations.
2016-12-18 14:55:08,331 - INFO - dtocean_installation.main - 7 possible solutions found.
2016-12-18 14:55:09,000 - WARNING - dtocean_installation.main - Lack of information on vessel fuel consumption. Fuel cost not considered 

## Examine the results

In [49]:
output_status = installation_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [50]:
economics_data = new_core.get_data_value(new_project, "farm.device_phase_installation_costs")
economics_data

,Equipment,Port,Vessel
Component,,,
Support Structure,148631.412213,258096.511544,2.174237e+06
Device,189500.768173,274935.732219,2.284921e+06


In [51]:
economics_data_breakdown = new_core.get_data_value(new_project, "farm.device_phase_installation_cost_breakdown")
economics_data_breakdown

{'Device': 2749357.3221885865, 'Support Structure': 2580965.115444631}

In [52]:
port = new_core.get_data_value(new_project, "farm.port")
port

'Portland port'

In [53]:
comp_cost = new_core.get_data_value(new_project, "farm.electrical_phase_installation_costs")
comp_cost

,Equipment,Port,Vessel
Component,,,
Dynamic Cables,313483.352648,373882.627449,3.051460e+06
Collection Points,191099.862796,92625.343431,6.425282e+05


In [54]:
comp_time = new_core.get_data_value(new_project, "farm.electrical_phase_installation_times")
comp_time

,Preparation,Operations,Transit,Waiting
Component,,,,
Dynamic Cables,50.004444,17.549848,177.118894,605.714286
Collection Points,97.000000,12.889265,104.168136,370.857143


In [55]:
economics_data_breakdown = new_core.get_data_value(new_project, "farm.electrical_phase_installation_time_breakdown")
economics_data_breakdown

{'Collection Points': 584.9145433099629, 'Dynamic Cables': 850.3874724828206}

In [56]:
comp_cost = new_core.get_data_value(new_project, "farm.mooring_phase_installation_costs")
comp_cost

,Equipment,Port,Vessel
Component,,,
Driven Piles,256950.510366,126864.705994,884831.843581
Pile Anchors,30454.877497,23183.145518,178193.432161


In [57]:
comp_time = new_core.get_data_value(new_project, "farm.mooring_phase_installation_times")
comp_time

,Preparation,Operations,Transit,Waiting
Component,,,,
Driven Piles,78.0,42.6949,236.725683,0.0
Pile Anchors,78.0,12.0000,65.757134,0.0


In [58]:
economics_data_breakdown = new_core.get_data_value(new_project, "farm.mooring_phase_installation_time_breakdown")
economics_data_breakdown

{'Driven Piles': 357.4205821833392, 'Pile Anchors': 155.75713403401136}

In [59]:
device_cost_breakdown = new_core.get_data_value(new_project, "farm.device_phase_cost_class_breakdown")
electrical_cost_breakdown = new_core.get_data_value(new_project, "farm.electrical_phase_installation_cost_breakdown")
mooring_cost_breakdown = new_core.get_data_value(new_project, "farm.mooring_phase_installation_cost_breakdown")

In [60]:
economics_data_breakdown = new_core.get_data_value(new_project, "farm.installation_phase_cost_breakdown")
economics_data_breakdown

{'Contingency': 1149588.0661553591,
 'Devices': 5330322.437633217,
 'Electrical Sub Systems': 4665079.708804152,
 'Mooring and Foundations': 1500478.5151162227}

In [61]:
economics_data_breakdown = new_core.get_data_value(new_project, "farm.installation_cost_class_breakdown")
economics_data_breakdown

{'Contingency': 1149588.0661553591,
 'Equipment': 1130120.7836942,
 'Port': 1149588.0661553594,
 'Vessel': 9216171.811704032}

In [62]:
device_time_breakdown = new_core.get_data_value(new_project, "farm.device_phase_time_class_breakdown")
device_time_breakdown

{'Operations': 30.0,
 'Preparation': 148.0,
 'Transit': 236.31620524259668,
 'Waiting': 597.0}

In [63]:
economics_data_breakdown = new_core.get_data_value(new_project, "farm.installation_phase_time_breakdown")
economics_data_breakdown

{'Devices': 1011.3162052425967,
 'Electrical Sub Systems': 1435.3020157927835,
 'Mooring and Foundations': 513.1777162173505}

In [64]:
economics_data_breakdown = new_core.get_data_value(new_project, "farm.installation_time_class_breakdown")
economics_data_breakdown

{'Operations': 115.13401206139258,
 'Preparation': 451.00444444444446,
 'Transit': 820.0860521754653,
 'Waiting': 1573.5714285714284}

In [65]:
economics_data_breakdown = new_core.get_data_value(new_project, "farm.installation_economics_data")
economics_data_breakdown

,Key Identifier,Cost,Quantity,Year
0,Electrical Sub Systems,4.665080e+06,1,0
1,Contingency,1.149588e+06,1,0
2,Devices,5.330322e+06,1,0
3,Mooring and Foundations,1.500479e+06,1,0


In [66]:
output_status = eco_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [67]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.capex_breakdown")
economics_data_breakdown

{'Devices': 2000000.0, 'Installation': 12645468.72770895}

In [68]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.capex_total")
economics_data_breakdown

14645468.72770895